In [1]:
import pandas as pd
import numpy as np
import LinearModels as LM
import matplotlib.pyplot as plt
import NonLinearModels_post as nlm

from scipy.stats import norm
from scipy.stats import genextreme

%load_ext autoreload
%autoreload 2


In [2]:
pd.options.display.float_format = '{:.12f}'.format
# Chose path
path = "../Sourcecode/"

In [3]:
# We begin by reading the csv file with the original dataset "choice_data"
choice_data = pd.read_csv(path + 'choice_data.csv', delimiter=';', encoding = 'unicode_escape')
choice_data.rename(columns = {'No. of registrations': 'Registrations'}, inplace = True)
# We create dummy variables for fuel
dummy_fuel = pd.get_dummies(choice_data['Fuel'], drop_first = False)
choice_data = choice_data.join(dummy_fuel)
choice_data

,Year,Make-model-year-fuel,Make,Model,Variant,Size segment,Fuel-size segment,Registrations,Shares,Prices (2015-DKK),...,Fuel,Engine effect (kW),Horsepower,Size (m3),Footprint (m²),Cost/km (DKK),fuelPrice (DKK/unit),Benzin,Diesel,El
0,2006,toyota-avensis-2006-Benzin,toyota,avensis,"1,8 STW MAN.",Small,BenzinSmall,5428,0.002156418000,403349.299100000004,...,Benzin,108.000000000000,139.307692300000,12.573440000000,8.223261538000,0.813669065000,11.310000000000,1,0,0
1,2006,volkswagen-passat-2006-Diesel,volkswagen,passat,"2,0 TDI",Large,DieselLarge,5378,0.002136554000,378504.672900000005,...,Diesel,103.000000000000,128.333333300000,12.734904000000,8.657173611000,0.681879195000,10.160000000000,0,1,0
2,2006,renault-clio-2006-Benzin,renault,clio,"1,2 8V 60 HK",Small,BenzinSmall,4518,0.001794896000,134344.626199999999,...,Benzin,43.000000000000,60.000000000000,8.872728000000,6.248400000000,0.677245509000,11.310000000000,1,0,0
3,2006,suzuki-swift-2006-Benzin,suzuki,swift,"1,5",Small,BenzinSmall,4406,0.001750401000,163536.214999999997,...,Benzin,68.000000000000,94.000000000000,9.379500000000,6.253000000000,0.734415584000,11.310000000000,1,0,0
4,2006,fiat-punto-2006-Benzin,fiat,punto,"1,2",Small,BenzinSmall,3743,0.001487007000,150287.383200000011,...,Benzin,48.000000000000,63.888888890000,9.483248000000,6.392844444000,0.689634146000,11.310000000000,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3377,2020,volkswagen-touareg-2020-Diesel,volkswagen,touareg,3.0 TDI V6 286 HK,Large,DieselLarge,35,0.000012800000,977674.081200000015,...,Diesel,210.000000000000,286.000000000000,16.619328000000,9.662400000000,0.643712575000,10.750000000000,0,1,0
3378,2020,volkswagen-golf-sportsvan-2020-Benzin,volkswagen,golf sportsvan,1.5 TSI EVO 150 DSG7,Large,BenzinLarge,818,0.000299839000,303593.810400000017,...,Benzin,110.000000000000,150.000000000000,12.411532000000,18.835600000000,0.583653846000,12.140000000000,1,0,0
3379,2020,volkswagen-up-2020-El,volkswagen,up,ELEKTRO MOTOR 82 Aut. 1 trins,Small,ElSmall,370,0.000135624000,153689.555099999998,...,El,61.000000000000,61.222222220000,8.855775289000,5.905600000000,0.306765224000,213.906561500000,0,0,1
3380,2020,volkswagen-t-cross-2020-Benzin,volkswagen,t-cross,1.0 TSI 115HK DSG7,Small,BenzinSmall,2969,0.001088290000,263941.972899999993,...,Benzin,85.000000000000,115.000000000000,11.429088000000,16.892100000000,0.541964286000,12.140000000000,1,0,0


In [4]:
# We investigate the number of car models each year 
year = 2006
for x in range(2006,2021):
    obs_count = choice_data.loc[(choice_data['Year'] == year)]
    print(len(obs_count))
    year += 1

186
210
215
198
217
225
232
242
248
246
246
239
234
230
214


In [5]:
# As we found that 2006 is the year with the least observations, we clean the datafile and makes sure all years
# include the same number of car models. 
year = 2006
for x in range(2006,2021):
     while len(choice_data.loc[choice_data['Year'] == year]) > len(choice_data.loc[choice_data['Year'] == 2006]):
          min_val = choice_data.loc[(choice_data['Year'] == year) & (choice_data['Fuel'] != 'El'), 'Registrations'].idxmin()
          choice_data.drop(index=min_val,inplace=True)
     year += 1
choice_data.to_csv(path + 'clogitchoice_data.csv')

In [6]:
# We investigate whether there is the same number of car models. 
year = 2006
for x in range(2006,2021):
    obs_count = choice_data.loc[(choice_data['Year'] == year)]
    print(len(obs_count))
    year += 1

186
186
186
186
186
186
186
186
186
186
186
186
186
186
186


In [7]:
# We create the market shares of every car based on the cleaned data.
total_reg = choice_data['Registrations'].sum(axis=0)
choice_data['MarketShare'] = choice_data['Registrations'] / total_reg 
choice_data.head()

,Year,Make-model-year-fuel,Make,Model,Variant,Size segment,Fuel-size segment,Registrations,Shares,Prices (2015-DKK),...,Engine effect (kW),Horsepower,Size (m3),Footprint (m²),Cost/km (DKK),fuelPrice (DKK/unit),Benzin,Diesel,El,MarketShare
0,2006,toyota-avensis-2006-Benzin,toyota,avensis,"1,8 STW MAN.",Small,BenzinSmall,5428,0.002156418000,403349.299100000004,...,108.000000000000,139.307692300000,12.573440000000,8.223261538000,0.813669065000,11.310000000000,1,0,0,0.001802670663
1,2006,volkswagen-passat-2006-Diesel,volkswagen,passat,"2,0 TDI",Large,DieselLarge,5378,0.002136554000,378504.672900000005,...,103.000000000000,128.333333300000,12.734904000000,8.657173611000,0.681879195000,10.160000000000,0,1,0,0.001786065369
2,2006,renault-clio-2006-Benzin,renault,clio,"1,2 8V 60 HK",Small,BenzinSmall,4518,0.001794896000,134344.626199999999,...,43.000000000000,60.000000000000,8.872728000000,6.248400000000,0.677245509000,11.310000000000,1,0,0,0.001500454321
3,2006,suzuki-swift-2006-Benzin,suzuki,swift,"1,5",Small,BenzinSmall,4406,0.001750401000,163536.214999999997,...,68.000000000000,94.000000000000,9.379500000000,6.253000000000,0.734415584000,11.310000000000,1,0,0,0.001463258463
4,2006,fiat-punto-2006-Benzin,fiat,punto,"1,2",Small,BenzinSmall,3743,0.001487007000,150287.383200000011,...,48.000000000000,63.888888890000,9.483248000000,6.392844444000,0.689634146000,11.310000000000,1,0,0,0.001243072272


In [8]:
# We create a market year variable, in which the market shares of all cars from each year are. 
year = 2006
choice_data['market_year'] = None
for x in range(2006,2021):
    total_year = choice_data.loc[(choice_data['Year'] == year), 'Registrations'].sum(axis=0)
    share_year = ((choice_data.loc[(choice_data['Year'] == year), 'Registrations']) / total_year)
    new_choice_data = pd.DataFrame({'market_year': share_year})
    choice_data.update(new_choice_data)
    year += 1

In [9]:
choice_data['market_year']

0      0.033475590202
1      0.033167229938
2      0.027863433407
3      0.027172706416
4      0.023083849323
            ...      
3353   0.000386870922
3378   0.003859273343
3379   0.001745637087
3380   0.014007558137
3381   0.015564477701
Name: market_year, Length: 2790, dtype: object

In [10]:
# We create the CO2 emission variable. 
choice_data["Fueleff"]=choice_data["Cost/km (DKK)"]/choice_data["fuelPrice (DKK/unit)"]
choice_data["Effi"]=choice_data["fuelPrice (DKK/unit)"]/choice_data["Cost/km (DKK)"]
choice_data["CO2"]=choice_data["Benzin"]*choice_data["Fueleff"]*2392+choice_data["Diesel"]*choice_data["Fueleff"]*2640+choice_data["El"]*choice_data["Fueleff"]*0
choice_data.head()

,Year,Make-model-year-fuel,Make,Model,Variant,Size segment,Fuel-size segment,Registrations,Shares,Prices (2015-DKK),...,Cost/km (DKK),fuelPrice (DKK/unit),Benzin,Diesel,El,MarketShare,market_year,Fueleff,Effi,CO2
0,2006,toyota-avensis-2006-Benzin,toyota,avensis,"1,8 STW MAN.",Small,BenzinSmall,5428,0.002156418000,403349.299100000004,...,0.813669065000,11.310000000000,1,0,0,0.001802670663,0.033475590202,0.071942446065,13.899999995698,172.086330988506
1,2006,volkswagen-passat-2006-Diesel,volkswagen,passat,"2,0 TDI",Large,DieselLarge,5378,0.002136554000,378504.672900000005,...,0.681879195000,10.160000000000,0,1,0,0.001786065369,0.033167229938,0.067114093996,14.899999991934,177.181208149606
2,2006,renault-clio-2006-Benzin,renault,clio,"1,2 8V 60 HK",Small,BenzinSmall,4518,0.001794896000,134344.626199999999,...,0.677245509000,11.310000000000,1,0,0,0.001500454321,0.027863433407,0.059880239523,16.699999999557,143.233532937931
3,2006,suzuki-swift-2006-Benzin,suzuki,swift,"1,5",Small,BenzinSmall,4406,0.001750401000,163536.214999999997,...,0.734415584000,11.310000000000,1,0,0,0.001463258463,0.027172706416,0.064935064898,15.400000008714,155.324675236782
4,2006,fiat-punto-2006-Benzin,fiat,punto,"1,2",Small,BenzinSmall,3743,0.001487007000,150287.383200000011,...,0.689634146000,11.310000000000,1,0,0,0.001243072272,0.023083849323,0.060975609726,16.400000008120,145.853658464368


In [11]:
# We create a subset with the variables we want to use later on in the model. 
choice_data['key'] = choice_data['Model'].str.replace('jaguar ', '-')
choice_data['key'] = choice_data['Make'] + '-' + choice_data['Model'] + '-' + choice_data['Year'].astype(str) + '-' + choice_data['Fuel']
choice_data['key'] = choice_data['key'].str.replace(' ', '-')
choice_data_subset = choice_data[['Fuel', 'Diesel', 'El', 'Make-model-year-fuel', 'Fuel-size segment', 'Registrations', 'Size segment', 'Year', 'Make', 'market_year', 'MarketShare', 'Weight (kg)', 'Engine effect (kW)', 'Prices (2015-DKK)', 'Cost/km (DKK)', 'Size (m3)', 'Horsepower','Ownership cost (DKK)', 'CO2', 'Effi']]
choice_data_subset.rename(columns={ choice_data_subset.columns[0]: "Fuel" })
choice_data_subset.insert(0, "ID", range(0, 0 + len(choice_data_subset)))

In [12]:
conditions = [
    (choice_data_subset['Fuel'] == 'El'),
    (choice_data_subset['Fuel'] == 'Diesel')]
choices = [0, 1]
choice_data_subset['Fuelchoice'] = np.select(conditions, choices, default=2)
choice_data_subset

C:\Users\Lasse\AppData\Local\Temp\ipykernel_9212\762087676.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choice_data_subset['Fuelchoice'] = np.select(conditions, choices, default=2)


,ID,Fuel,Diesel,El,Make-model-year-fuel,Fuel-size segment,Registrations,Size segment,Year,Make,...,Weight (kg),Engine effect (kW),Prices (2015-DKK),Cost/km (DKK),Size (m3),Horsepower,Ownership cost (DKK),CO2,Effi,Fuelchoice
0,0,Benzin,0,0,toyota-avensis-2006-Benzin,BenzinSmall,5428,Small,2006,toyota,...,1820,108.000000000000,403349.299100000004,0.813669065000,12.573440000000,139.307692300000,346885.205499999982,172.086330988506,13.899999995698,2
1,1,Diesel,1,0,volkswagen-passat-2006-Diesel,DieselLarge,5378,Large,2006,volkswagen,...,2170,103.000000000000,378504.672900000005,0.681879195000,12.734904000000,128.333333300000,352803.014000000025,177.181208149606,14.899999991934,1
2,2,Benzin,0,0,renault-clio-2006-Benzin,BenzinSmall,4518,Small,2006,renault,...,1425,43.000000000000,134344.626199999999,0.677245509000,8.872728000000,60.000000000000,118899.032099999997,143.233532937931,16.699999999557,2
3,3,Benzin,0,0,suzuki-swift-2006-Benzin,BenzinSmall,4406,Small,2006,suzuki,...,1475,68.000000000000,163536.214999999997,0.734415584000,9.379500000000,94.000000000000,135059.819400000008,155.324675236782,15.400000008714,2
4,4,Benzin,0,0,fiat-punto-2006-Benzin,BenzinSmall,3743,Small,2006,fiat,...,1590,48.000000000000,150287.383200000011,0.689634146000,9.483248000000,63.888888890000,135188.422499999986,145.853658464368,16.400000008120,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3353,2785,Diesel,1,0,dacia-duster-2020-Diesel,DieselSmall,82,Small,2020,dacia,...,1848,85.000000000000,170224.371400000004,0.401119403000,13.202280000000,103.750000000000,171706.721699999995,98.507462690233,26.799999999003,1
3378,2786,Benzin,0,0,volkswagen-golf-sportsvan-2020-Benzin,BenzinLarge,818,Large,2020,volkswagen,...,1930,110.000000000000,303593.810400000017,0.583653846000,12.411532000000,150.000000000000,272550.640300000028,114.999999969687,20.800000005483,2
3379,2787,El,0,1,volkswagen-up-2020-El,ElSmall,370,Small,2020,volkswagen,...,1530,61.000000000000,153689.555099999998,0.306765224000,8.855775289000,61.222222220000,138192.747299999988,0.000000000000,697.297296971315,0
3380,2788,Benzin,0,0,volkswagen-t-cross-2020-Benzin,BenzinSmall,2969,Small,2020,volkswagen,...,1760,85.000000000000,263941.972899999993,0.541964286000,11.429088000000,115.000000000000,237167.811500000011,106.785714342010,22.399999988191,2


In [13]:
# We make sure the market year variable is of type float and not object. 
choice_data_subset['market_year'] = choice_data_subset['market_year'].astype(str).astype(float)

C:\Users\Lasse\AppData\Local\Temp\ipykernel_9212\2204368512.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choice_data_subset['market_year'] = choice_data_subset['market_year'].astype(str).astype(float)


In [14]:
# We create a csv file with the data we use in the models produced in the jupyter notebook "ConditionalLogit"
# and to produced graphs in the notebook "Graphs"
choice_data_subset.rename(columns = {'Make-model-year-fuel': 'Key'}, inplace = True)
choice_data_subset.head()
choice_data_subset.to_csv(path + 'clogit__data_subset.csv')

C:\Users\Lasse\AppData\Local\Temp\ipykernel_9212\3777953804.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  choice_data_subset.rename(columns = {'Make-model-year-fuel': 'Key'}, inplace = True)
